<a href="https://colab.research.google.com/github/serereuk/Tobigo_renewal/blob/master/Vanilla_PG_in_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
import numpy as np
import gym
#tf.keras.backend.set_floatx('float64')

In [16]:
agent_input = tf.keras.Input(shape = 4, name = 'state')
x = tf.keras.layers.Dense(32, activation=tf.nn.relu)(agent_input)
output = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(x)
agent = tf.keras.Model(agent_input, output, name = 'agent')
optim = tf.keras.optimizers.Adam(lr = 0.01)
losses = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
agent.summary()

Model: "agent"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
state (InputLayer)           [(None, 4)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                160       
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 66        
Total params: 226
Trainable params: 226
Non-trainable params: 0
_________________________________________________________________


In [0]:
def discount(reward, gamma=0.8):
    discounted = []
    R = 0
    for i in reward[::-1]:
        R = i + gamma*R
        discounted.append(R)
    return discounted[::-1]

In [0]:
env = gym.make('CartPole-v0')
data = []
score = []
gamma = 0.98
print_interval = 100

gradBuffer = agent.trainable_variables
for ix,grad in enumerate(gradBuffer):
  gradBuffer[ix] = grad * 0

for n_epi in range(2000):
    ep_memory = []
    ep_score = 0
    s = env.reset()
    done = False
    while not done:
        s = s.reshape(1,4)
        with tf.GradientTape() as tape:
            prob = agent(s)
            a = tf.random.categorical(prob, 1).numpy().squeeze()
            loss = losses([a], prob)
        s, r, done, info = env.step(a)
        ep_score += r
        if done:
            r -= 10
        grads = tape.gradient(loss, agent.trainable_variables)
        ep_memory.append([grads, r])

    ep_memory = np.array(ep_memory)
    ep_memory[:,1] = discount(ep_memory[:,1])
    score.append(ep_score)

    for grads, r in ep_memory:
        for ix, grad in enumerate(grads):
            gradBuffer[ix] += grad * r

    optim.apply_gradients(zip(gradBuffer, agent.trainable_variables))
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0

    if n_epi % print_interval == 0:
        print('Episode: ', n_epi, "Score: ", np.mean(score[-100:]))

        


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Episode:  0 Score:  27.0
Episode:  100 Score:  47.17
Episode:  200 Score:  82.08
Episode:  300 Score:  73.78
Episode:  400 Score:  88.65


In [9]:
ep_memory[:,1]

array([-9.0, -6.2, -3.960000000000001, -2.168000000000001,
       -0.7344000000000008, 0.4124799999999993, 1.3299839999999994,
       2.0639871999999997, 2.65118976, 3.120951808], dtype=object)